In [ ]:
%load_ext autoreload 
%autoreload 2 

In [ ]:
import random
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # for subplots
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn.objects as so
import tensorflow as tf

from src.utilities.plot_helpers import generate_random_colors
from src.modeling.experiments.binned_trial_experiment_01 import split_data

from src.utilities.pandas_helpers import get_features

## Experimental conditions: 
* 2 layer LSTM 
* trained with 128 units

#### Load Model

In [ ]:
# directory paths
DATA_PATH = '/projects/p31961/gaby_data/aggregated_data/data_pipeline_full_dataset/datasets/full_dataset.parquet.gzip'
PATH_TO_MODELS = '/projects/p31961/ENIGMA/results/experiments/binned_trial_experiment_01/models'
control_path = os.path.join(PATH_TO_MODELS, 'control')
no_bins_path = os.path.join(PATH_TO_MODELS, 'dropped_binned_trials')


In [ ]:

control_model = tf.keras.models.load_model(control_path)
dropped_bins_model = tf.keras.models.load_model(no_bins_path)
control_model.compile(optimizer='adam', loss='mse', metrics=[
                  'mae', 'mse', 'mape', 'cosine_similarity'])
dropped_bins_model.compile(optimizer='adam', loss='mse', metrics=[
                  'mae', 'mse', 'mape', 'cosine_similarity'])


models = [control_model, dropped_bins_model]

# Model Summary

In [ ]:
for model in models:
    print(model.summary())

#### load training and testing data

In [ ]:
data= pd.read_parquet(DATA_PATH)
data.head()

In [ ]:
# split data for control
target='signal'
control_features = get_features(data, target)
no_bin_features = control_features.copy()
no_bin_features.remove('learning_phase')

X_train, y_train, X_val, y_val, X_test, y_test = split_data(data, control_features, target, day_cut_off=6, feature_to_drop=None)

    # split data for dropped binned trials
X_train_dropped, y_train_dropped, X_val_dropped, y_val_dropped, X_test_dropped, y_test_dropped = split_data(
data, no_bin_features, target, day_cut_off=6, feature_to_drop='learning_phase')

In [ ]:
y_control_pred = control_model.predict(X_test)
y_dropped_pred = dropped_bins_model.predict(X_test_dropped)

# Full dataset

## Validation check

In [ ]:
data_directory = os.path.dirname(DATA_PATH)

### control dataset with inference

In [ ]:
control_inference_data= X_test.assign(predicted_signal= y_control_pred,
                                              true_signal=y_test)
control_inference_data.to_parquet(os.path.join(data_directory, 'control_inference_data.parquet.gzip'), compression='gzip')

In [ ]:
control_inference_data.head()

In [ ]:
#load data if previously saved
control_inference_data = pd.read_parquet(os.path.join(data_directory, 'control_inference_data.parquet.gzip'))

In [7]:
events = ['event_cue','event_avoid', 'event_escape', 'event_shock']
sensors = ['sensor_D1', 'sensor_D2', 'sensor_DA']

def format_for_plotting(df):
        cols_to_merge = ['true_signal', 'predicted_signal']
        cols = ['time',
                'mouse_id',
                'day',
                'trial',
                'learning_phase',
                'latency',
                'event',
                'sensor',
                'male']
        return (
        control_inference_data
        .assign(event = lambda df_:df_[events].idxmax(axis=1).str.replace('event_', '').astype('category'),
                sensor = lambda df_: df_[sensors].idxmax(axis=1).str.replace('sensor_', '').astype('category'), 
                male = lambda df_: df_.sex_M.astype(bool)
                )
        .drop(columns=events+sensors + ['sex_M'])
        .melt(id_vars=cols, 
              value_vars=cols_to_merge, 
              var_name='signal_type', 
              value_name='signal')
        .assign(signal_type = lambda df_: df_.signal_type.str.replace('_signal', '').astype('category'))
        )
    
con_data= format_for_plotting(control_inference_data)
con_data.head()


,time,mouse_id,day,trial,learning_phase,latency,event,sensor,male,signal_type,signal
0,-25.000000,0,7,0,0,0.0,cue,D1,False,true,0.145303
1,-24.901531,0,7,0,0,0.0,cue,D1,False,true,-0.897972
2,-24.803064,0,7,0,0,0.0,cue,D1,False,true,-1.039647
3,-24.704596,0,7,0,0,0.0,cue,D1,False,true,-0.414173
4,-24.606127,0,7,0,0,0.0,cue,D1,False,true,0.291662


In [ ]:
# function to plot data for a given query of sex and sensor
def generate_plot(df, query, sensor, sex, palette):
    # cmap = sns.light_palette(palette_color, as_cmap=True)
    # palette = [cmap(x) for x in np.linspace(0, 1, 2)]
    
    title = f'{sex} | {sensor}'
    grid = (sns.FacetGrid(df.query(query),
                         row='event',
                         col= 'signal_type',
                         hue='learning_phase', 
                         palette=palette, 
                         sharex=True, 
                         sharey=True, 
                         margin_titles=True)
            .map(sns.lineplot, 'time','signal')
            .set_titles(row_template = '{row_name}', col_template = '{col_name}')
            .add_legend(title = 'Learning Phase', bbox_to_anchor=(1.2, 0.5), loc='center right')
    )
    
    grid.fig.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()
    return grid

In [ ]:
    
female_DA = "sensor == 'DA' & male == False"
female_DA_plot = generate_plot(con_data, female_DA, sensor='DA', sex='female', palette='Greens')

In [ ]:
male_DA = "sensor == 'DA' & male == True"
male_DA_plot = generate_plot(con_data, male_DA, sensor='DA', sex='male', palette='Greens')

In [ ]:
female_D1 = "sensor == 'D1' & male == False"  
female_D1_plot = generate_plot(con_data, female_DA, sensor = 'D1', sex = 'female', palette='Blues')

In [ ]:

male_D1 = "sensor == 'D2' & male == True"
male_D1_plot = generate_plot(con_data, male_D1, sensor = 'D1', sex = 'male', palette='Blues')

In [ ]:
female_D2 = "sensor == 'D2' & male == False"
female_D2_plot = generate_plot(con_data, female_D2, sensor = 'D2', sex = 'female', palette='Reds')

In [ ]:
male_D2 = "sensor == 'D2' & male == True"
male_D2_plot = generate_plot(con_data, male_D2, sensor = 'D2', sex = 'male', palette='Reds')

### no trial bins data with inference

In [ ]:
no_bins_inference_data= X_test_dropped.assign(predicted_signal= y_dropped_pred,
                                              true_signal=y_test_dropped)
no_bins_inference_data.to_parquet(os.path.join(data_directory, 'no_bins_inference_data.parquet.gzip'), compression='gzip')

In [ ]:
no_bins_inference_data.head()

# Conclusions 



# Next steps

In [ ]:
sns.palplot(sns.color_palette("Greens", 3))